In [1]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

from beamtest_analysis_helper import DecodeBinary
from natsort import natsorted
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
path_to_loop_dir = "GTX_320Mbps_Board0_selfTrigger/loop_0" # Change this path
files = natsorted(list(Path(path_to_loop_dir).glob('T*bin')))
files = files[1:] # Ignore first binary file

In [ ]:
decoder = DecodeBinary(
    firmware_key = 0b0001,
    board_id = [0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
    file_list = files,
    save_nem = None,
    skip_filler = False,
    skip_event_df = True,
    skip_crc_df = True,
)
df, _, _, filler_df = decoder.decode_files()

del df

In [ ]:
filtered_df = filler_df.loc[filler_df['type'] == b'40']
filtered_df.reset_index(drop=True, inplace=True)
filtered_df

In [ ]:
del filler_df

In [ ]:
fig = plt.figure(figsize=(14, 10))
plt.plot(filtered_df['idx'].values*0.025, filtered_df['events'].values)
plt.xlabel('Time [s]')
plt.ylabel('Interval')

In [ ]:
denominator = filtered_df.shape[0] * 0.025 * 1.024**2
denominator

In [ ]:
prescale_factor = 2048 # change prescale factor
numerator = (filtered_df['last_event'].values[-1] - filtered_df['prev_event'].values[0]) * prescale_factor
numerator

In [ ]:
trigger_rate = numerator/denominator
trigger_rate_in_kHz = trigger_rate/1e3
print(f"{trigger_rate_in_kHz:.2f} kHz")